In [12]:
%run search_functions.ipynb

In [1]:
import pandas as pd
import pysentiment2.renault as ren
import yfinance as yf

In [4]:
# get BTC data
btc_price = yf.Ticker("BTC-USD").history(period="max")["Close"][:pd.to_datetime('2019-01-01')]
btc_price_df = yf.Ticker("BTC-USD").history(period="max").reset_index()
btc_price_df['close_return'] = get_pct_change(btc_price_df, 'Close')
btc_price_df['vol_return'] = get_pct_change(btc_price_df, 'Volume')

#get CRIX data 
PATH_crix = "C://Users//jackl//OneDrive//Documents//Harvard//Research//Greenwood//crypto//crix.json"
crix = pd.read_json(PATH_crix)
crix['close_return'] = get_pct_change(crix, 'price')
crix.columns = ['Date', 'Close', 'close_return']
crix['Date'] = pd.to_datetime(crix['Date'])
crix_df = crix.set_index('Date')

In [5]:
# get twitter data
PATH_twitter = "C://Users//jackl//OneDrive//Documents//Harvard//Research//Greenwood//crypto//crypto_master_tweets.xlsx"
twitter_data = pd.read_excel(PATH_twitter).sort_values('date')
fdata_twitter = pd.DataFrame({'full_text': twitter_data['tweet_text'], 
                              'date': twitter_data['date'],
                              'reactions': twitter_data['reactions']})
fdata_twitter = get_flags(fdata_twitter)

In [6]:
# append sentiment scores
RN = ren.RN()
sent_score_list = []
for i in range(len(fdata_twitter)): 
    tokens = RN.tokenize(fdata_twitter.iloc[i]['full_text'])
    sent_score_list.append(RN.get_score(tokens)['Polarity'])
fdata_twitter['sent_score'] = sent_score_list

In [13]:
#get regression data
reg_data = get_reg_data(fdata_twitter, crix, 'W')
reg_data.to_stata("C://Users//jackl//OneDrive//Documents//Harvard//Research//Greenwood//crypto_v2_crix.dta")

In [20]:
# Proquest data 
# data1 = pd.read_excel('C://Users//jackl//OneDrive//Documents//Harvard//Research//Greenwood//crypto//ProQuestDocuments-2020-10-04.xls')
# data2 = pd.read_excel('C://Users//jackl//OneDrive//Documents//Harvard//Research//Greenwood//crypto//ProQuestDocuments-2020-10-04 (1).xls')

# data = pd.concat([data1, data2], join='inner')
# data['pubdate'] = pd.to_datetime(data['pubdate'])

# start_date = pd.to_datetime("2015-01-01")
# data = data[data['pubdate'] > start_date]
# data = data.sort_values(['pubdate']).reset_index(drop=True)
# data['full_text'] = data['Title'].astype(str) + " " + data['Abstract'].astype(str)

# fdata = pd.DataFrame({"date": data['pubdate'], "full_text": data['full_text']})

# RN = ren.RN()
# sent_score_list = []
# for i in range(len(fdata)): 
#     tokens = RN.tokenize(fdata.iloc[i]['full_text'])
#     sent_score_list.append(RN.get_score(tokens)['Polarity'])

# fdata['sent_score'] = sent_score_list